In [169]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
from plotly.tools import FigureFactory as FF
import re
import os
from datetime import datetime, timedelta
os.getcwd()
df = pd.DataFrame()

In [170]:
df = pd.read_csv('./activityQ.csv', nrows = 21111)
re1 = re.compile(r'(?<=\.).+')
df.columns = [re1.findall(i)[0] for i in list(df.columns.to_numpy())]


In [172]:
df.columns

Index(['wms_warehouse_id', 'user_id', 'activity_type', 'activity_sub_type',
       'activity_start_time', 'activity_end_time', 'lpn', 'sku_code',
       'order_id', 'from_location', 'to_location', 'qty', 'create_time',
       'internal_container_num', 'internal_id', 'activity_id', 'hh',
       'data_source', 'wms_company_id', 'transaction_type', 'transaction_code',
       'standard_activity_type', 'inc_day', 'src'],
      dtype='object')

In [171]:
df['activity_end_time'] = pd.to_datetime(df['activity_end_time'])
df['activity_start_time'] = pd.to_datetime(df['activity_start_time'])

In [173]:
"""
d单个activity的耗费时间.
"""
df['time_gap_inner'] = df['activity_end_time']-df['activity_start_time']
df['time_gap_inner'] = df['time_gap_inner'].fillna(
    timedelta(0)
    ).apply(timedelta.total_seconds)
df['time_gap_inner'] = [0 if i< 0 else i for i in df['time_gap_inner']]

In [222]:
"""
前一个activity到后一个activity相差的时间\.
"""
df['time_gap_outer'] = df.sort_values('activity_start_time')\
    .groupby(['user_id','activity_id','inc_day'])['activity_start_time'].diff()
df['time_gap_outer'] = df['time_gap_outer'].fillna(timedelta(0)).apply(timedelta.total_seconds)
df['time_gap_outer'] = [0 if i< 0 else i for i in df['time_gap_outer']]

In [224]:
"""
time gap of the day.
"""
mid1 = df.groupby(['inc_day', 'user_id']).agg(
    a = ('activity_start_time','min'), 
    b  = ('activity_start_time', 'max')).reset_index()

mid1['time_gap_today'] = mid1[['a', 'b']].\
    diff(axis = 1).drop('a', axis =1 )['b'].apply(timedelta.total_seconds)

df = df.merge(mid1.drop(['a', 'b'], axis = 1), on = ['inc_day', 'user_id'], how = 'left')


In [ ]:
df[['activity_type', 'activity_sub_type','user_id','inc_day',
       'activity_start_time', 'activity_end_time','time_gap_inner',
       'time_gap_outer']].sort_values(['user_id','inc_day'])

In [ ]:
df[['ou_code', 'ou_name', 'bg_code', 'bg_name_cn', 'customer_id',
       'customer_name',  'operation_day', 'hour',
       'wms_warehouse_id', 'user_id', 'activity_type', 'activity_sub_type',
       'activity_start_time', 'activity_end_time', 'lpn', 'sku_code',
       'order_id', 'from_location', 'to_location', 'qty', 'create_time',
       'activity_id', 'inc_day','act_tme_diff_row']].to_csv('./activi_for.csv', index=None, encoding='utf_8_sig')

In [ ]:
df['act_tme_diff_row'].hist()

In [ ]:
df['act_tme_diff'].hist()